# **ETL**

## Objectives

- Load "insurance.csv" dataset downloaded from Kaggle 
- Briefly describe the dataset
- Clean the dataset i.e. deal with missing, duplicated, inconsistent or invalid values 
- Convert data into correct type and format
- Aggregate data to summarise charge prediction
- Save the modified dataset 

## Inputs

- insurance.csv
- Python libraries: Numpy; Pandas; Feature Engine; Matplotlib; Seaborn; Plotly

## Outputs

* cleaned and transformed version: "transformed_insurance.csv"

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\sonia\\Documents\\VS Studio Projects\\Hackathon2_project_team3\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\sonia\\Documents\\VS Studio Projects\\Hackathon2_project_team3'

# Load the Dataset

Before loading the dataset, I will install the required Python libraries.

In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from feature_engine.imputation import CategoricalImputer
import matplotlib.pyplot as plt
import seaborn as sns

Now I will load the dataset which is saved in this directory under Dataset/ Raw.

In [5]:
df = pd.read_csv("Dataset/Raw/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


---

# Initial Exploration and Clening

Next, I will look at the basic characteristics of the dataset including, size, data types and summary statistics of the dataset.

In [6]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


This shows that the DataFrame has 7 columns and 1338 rows, which should correspond to 1338 separate individuals. I will check for duplicate rows when I clean the dataset. 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


I can see that we have the expected data types, with **age** and number of **children** columns being integers; **bmi** and **charges** being floats; and the categorical data **sex**, **smoker** and **region** being objects. We will encode the categorical data at a later point to aid in exploratory data analysis (EDA).

I will look at summary statistics on the numerical data next. 

In [8]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


I can see that **age**, **bmi** and **chidren** columns have valid ranges, for example, a minimum age of 18 years old and maximum of 64.

I am going to limit the **charges** column to 2 decimal places, as this is the standard way to view/ represent currency.

In [9]:
df["charges"] = df["charges"].round(2)

In [10]:
df["charges"]

0       16884.92
1        1725.55
2        4449.46
3       21984.47
4        3866.86
          ...   
1333    10600.55
1334     2205.98
1335     1629.83
1336     2007.94
1337    29141.36
Name: charges, Length: 1338, dtype: float64

Next, I will look at the unique values of the categorical data. While the column **children** is numerical, I will change it to "object" for this exercise, before changing it back to "int". 

In [11]:
df["children"] = df["children"].astype("object")
df["children"].dtype

dtype('O')

In [12]:
for col in df.select_dtypes("object"):
    print(f'{col}: {df[col].unique()}')

sex: ['female' 'male']
children: [0 1 3 2 5 4]
smoker: ['yes' 'no']
region: ['southwest' 'southeast' 'northwest' 'northeast']


I can see that there are only expected and valid values for each of these categorical variables. 

I would also like to see the number of counts for each value of the categorical variables, both as the absolute number and proportion of the total.

In [13]:
for col in df.select_dtypes("object"):
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True))

sex
male      676
female    662
Name: count, dtype: int64
sex
male      0.505232
female    0.494768
Name: proportion, dtype: float64
children
0    574
1    324
2    240
3    157
4     25
5     18
Name: count, dtype: int64
children
0    0.428999
1    0.242152
2    0.179372
3    0.117339
4    0.018685
5    0.013453
Name: proportion, dtype: float64
smoker
no     1064
yes     274
Name: count, dtype: int64
smoker
no     0.795217
yes    0.204783
Name: proportion, dtype: float64
region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64
region
southeast    0.272048
southwest    0.242900
northwest    0.242900
northeast    0.242152
Name: proportion, dtype: float64


I can see that these data appear valid. There are ~20 % smokers and ~80 % non-smokers; there is almost equal representation from all 4 regions; and ~40 % of the people have no children and this percentage decreases as the number of chidrem increases.

I will return **children** back to the "int" data type.

In [14]:
df["children"] = df["children"].astype("int")
df["children"].dtype

dtype('int32')

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
